This file shows how t crreate embedings, store it in a vector index, and create RAG pipeline

How to create embedings?

In [10]:
from dotenv import load_dotenv

In [11]:
load_dotenv()

True

In [12]:
import os

In [13]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [ ]:
# Import the HuggingFaceEmbeddings class and create an instance of it
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\HP\AppData\Local\Temp\ipykernel_23496\4286477000.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [15]:
embeddings.embed_query("hello AI")

[-0.033388227224349976,
 0.03453974053263664,
 0.05947453901171684,
 0.05928609520196915,
 -0.06353536248207092,
 -0.06819580495357513,
 0.08823321014642715,
 0.03444075956940651,
 -0.03278519585728645,
 -0.015814989805221558,
 0.020981740206480026,
 -0.01834029145538807,
 -0.039832137525081635,
 -0.08047075569629669,
 -0.014469224028289318,
 0.0332648791372776,
 0.014259224757552147,
 -0.03404993936419487,
 -0.142915740609169,
 -0.023083269596099854,
 -0.021380074322223663,
 0.0026335334405303,
 -0.047292742878198624,
 -0.010752737522125244,
 -0.06866798549890518,
 0.03112504631280899,
 0.0759458988904953,
 0.001128267147578299,
 0.011631963774561882,
 -0.03603918477892876,
 0.044837627559900284,
 0.018390750512480736,
 0.12672799825668335,
 -0.0013597796205431223,
 0.008206719532608986,
 0.06909963488578796,
 -0.08076353371143341,
 -0.05841311067342758,
 0.053754501044750214,
 0.02622760646045208,
 -0.006828587502241135,
 -0.05635837838053703,
 0.003292947541922331,
 -0.0725017860531

How to calculate Cosine Similarity?

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Create some sample documents
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [ ]:
# sample query
my_query="Narendra modi is prime minister of india?"

In [ ]:
# Embed the documents
document_embedding=embeddings.embed_documents(documents)

In [25]:
document_embedding

[[0.11998698115348816,
  -0.02130264602601528,
  -0.04288088157773018,
  0.06645584106445312,
  -0.06435240805149078,
  -0.04424865171313286,
  0.022408459335565567,
  -0.04987306892871857,
  -0.023437613621354103,
  -0.033972058445215225,
  -0.014048044569790363,
  -0.0606592632830143,
  -0.003906767349690199,
  -0.017782030627131462,
  -0.04797105863690376,
  -0.0666816383600235,
  0.004103203769773245,
  -0.013092738576233387,
  0.0443977490067482,
  0.022350654006004333,
  0.009459571912884712,
  -0.020564530044794083,
  -0.00033557237475179136,
  -0.005685783457010984,
  0.05558699369430542,
  0.025123203173279762,
  -0.0028171101585030556,
  0.00875901523977518,
  0.0032552604097872972,
  -0.015963425859808922,
  0.014263702556490898,
  -0.1122085303068161,
  0.08968561142683029,
  -0.031083758920431137,
  -0.024223830550909042,
  0.006152115762233734,
  0.08058713376522064,
  0.018249982967972755,
  0.05568313971161842,
  0.01670265942811966,
  0.015896005555987358,
  0.00034107

In [ ]:
# Embed the query 
query_embedding=embeddings.embed_query(my_query)

In [27]:
len(document_embedding),len(query_embedding)

(3, 384)

In [ ]:
# Calculate cosine similarity between the query and the documents
cosine_similarity(document_embedding,[query_embedding])

array([[0.1175667 ],
       [0.34324559],
       [0.81413232]])

How to calculate Eclidian Distance(L2 distance)?

In [32]:
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# Calculate euclidean distance between the query and the documents
euclidean_distances([query_embedding], document_embedding)

array([[1.32848282, 1.14608418, 0.60970104]])

How to store embedings in vector store?

In [34]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [35]:
index=faiss.IndexFlatL2(384)


In [36]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023B7454BB10> >

In [37]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [38]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['6696ca42-a97f-44cd-a13a-da7c62e1719e',
 'c40f6167-4eec-4698-88ee-6a467fdfcf4a',
 'd62cf803-60aa-4b8b-a11e-a868bee32996']

In [39]:
vector_store.index_to_docstore_id

{0: '6696ca42-a97f-44cd-a13a-da7c62e1719e',
 1: 'c40f6167-4eec-4698-88ee-6a467fdfcf4a',
 2: 'd62cf803-60aa-4b8b-a11e-a868bee32996'}

In [40]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [41]:
results

[Document(metadata={}, page_content='AI is powerful'),
 Document(metadata={}, page_content='AI is future'),
 Document(metadata={}, page_content='Dogs are cute')]

In [42]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [43]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [44]:
vector_store.add_documents(documents=documents)

['bf4a5fe5-6183-4454-8061-b51231431b84',
 '8f2f78d3-22e7-4262-8ce7-e4bddcd646ce',
 'd015b941-14a0-4096-9a70-be761003b8c8',
 '4f5a9d6e-2dd4-4ce8-a198-cf9e3acc2546',
 '86dfd59d-377a-4b77-bd0c-164e8b41732a',
 '74ca48c0-da2d-4ba9-889a-060decbab175',
 '875bf5e6-34c4-4db9-8386-d034062f0f50',
 '59e95a43-b291-4ee2-9477-8f4f3f76abdf',
 '35ca5788-1cfa-4e6d-bae0-dc493f7213ac',
 '869bd61a-e16d-4b0e-b0e0-3e52a8745968']

In [45]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [48]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"tweet"}
    
)

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [49]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [50]:
result[0].metadata

{'source': 'news'}

In [51]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [54]:
retriever=vector_store.as_retriever(search_kwargs={"k": 2})

In [55]:
retriever.invoke("What is LangChain?")

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [56]:
vector_store.save_local("today's class faiss index")

In [57]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [58]:
new_vector_store.similarity_search("langchain")

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

RAG Using PDF

In [ ]:
# load the pdf file and split it into chunks
from langchain_community.document_loaders import PyPDFLoader

In [60]:
FILE_PATH=r"D:\2026\Krish Naik\Agentic AI My Practice\Traditional RAG- 1\llama2.pdf"

In [61]:
loader =  PyPDFLoader(FILE_PATH)

In [62]:
len(loader.load())

77

In [63]:
pages = loader.load()

In [66]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)



# Why Use Async? # Because:

# If you're loading:
# Web pages
# PDFs
# APIs
# Large files

# Async allows:
# Non-blocking execution
# Better performance
# Faster IO operations

<!-- Why Use Async?

Because:

If you're loading:

Web pages

PDFs

APIs

Large files

Async allows:

Non-blocking execution

Better performance

Faster IO operations -->

In [67]:
pages

[Document(metadata={'source': 'D:\\2026\\Krish Naik\\Agentic AI My Practice\\Traditional RAG- 1\\llama2.pdf', 'page': 0}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jia

In [68]:
len(pages)

77

In [ ]:
# split the pages into chunks 
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [70]:
split_docs = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [74]:
split_docs = split_docs.split_documents(pages)

In [75]:
len(split_docs)

615

In [ ]:
# create a vector store and add the documents to it
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [77]:
vector_store.add_documents(documents=split_docs)

['6742c463-9916-472d-9630-9b5909b612fd',
 'dba98040-aa34-4361-8ef2-4adc54cf9fe3',
 '55f40689-175c-4efd-895a-34abce09b56c',
 'dc834fff-6c6e-48c5-983d-51704636a63b',
 '9ad6023d-d3f1-4bc7-8280-a59ee13c81fc',
 'a2e4b205-bd65-458f-9ba7-c70496892b6c',
 '05f64816-6ebe-466c-b366-7f87cbef58ce',
 'e04183cd-3b70-4c7c-9e15-db572c4c2cf9',
 '8a119a74-7114-40b6-9d18-b58231107383',
 '385806a6-ea64-4d64-8526-c649218ffb2a',
 '4a3b5fce-e846-4d15-9d1b-4bddc4af7e66',
 'b72ae5f0-5c97-4814-8091-9c38e315a7c1',
 '04601739-ad39-49cd-882d-6efdc97df17c',
 '09f1d31d-6170-4536-9fc7-0d57c98ba6f9',
 '8a18b835-020f-46e4-825d-065c4b6b381e',
 '5759443f-90bb-4948-a336-befba07e9fe8',
 '4e5096c6-e0ce-47e0-b7ab-896fec0a557e',
 '65424616-f245-480e-81de-a2d04b681443',
 'f826991f-0a75-42f3-8bcf-053ce79c4b81',
 'ea088fbd-04df-44d5-9a23-caa12a640f5c',
 '3c53aa3a-bd85-42d3-802f-20bbb30f82a4',
 'cd3db901-32e9-4084-840c-8ab15b5cee6c',
 '76471a9c-fad8-4f97-b8f0-fd28a39ba743',
 '1d991dad-763a-4ebb-814c-b611502b137a',
 'c315c96e-ce26-

In [ ]:
# create a retriever from the vector store
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [79]:
retriever.invoke("What is Llama model?")

[Document(metadata={'source': 'D:\\2026\\Krish Naik\\Agentic AI My Practice\\Traditional RAG- 1\\llama2.pdf', 'page': 3}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(metadata={'source': 'D:\\2026\\Krish Naik\\Agentic AI My Practice\\Traditional RAG- 1\\llama2.pdf', 'page': 76}, page_content='models-and-libraries/llama-downloads/\nWhere to send com-\nments\nInstructions on how to provide feedback or comments on the model can be\nfound in the model README, or by opening an issue in the GitHub repository\n(https://github.com/facebookresearch/llama/).\nIntended Use\nIntended Use CasesLlama 2is intended for commercial and research use in English. Tuned models\nare intended for assistant-like chat, whereas pretrained models can be adapt

In [88]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [ ]:
# load model 
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [ ]:
# load promt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [111]:
import pprint

In [112]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [113]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [114]:
#context(retriever),prompt(hub),model(google),parser(langchain)

In [115]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [116]:
#RAG Pipeline with LangChain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [119]:
response = rag_chain.invoke("What is Llama model?")    
print(response) 

Llama models are a collection of large language models (LLMs) developed for commercial and research use in English. Tuned versions are intended for assistant-like chat, while pretrained models can be adapted for various natural language generation tasks. Llama 2 models, for example, were trained between January and July 2023 and outperform earlier Llama 1 models.


In [118]:
# import google.generativeai as genai
# import os

# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# for m in genai.list_models():
#     print(m.name)

In [ ]:
# display the response in markdown format
from IPython.display import Markdown, display
display(Markdown(response))

Llama models are a collection of large language models (LLMs) developed for commercial and research use in English. Tuned versions are intended for assistant-like chat, while pretrained models can be adapted for various natural language generation tasks. Llama 2 models, for example, were trained between January and July 2023 and outperform earlier Llama 1 models.